# 🔄 1D Convolutional Neural Networks для табличных данных

**Phase 2: Deep Learning Basics - Step 2**

---

## 🎯 Цели ноутбука

1. **Понять 1D convolutions** и их применение
2. **Адаптировать CNN для табличных данных**
3. **Сравнить с MLP** из предыдущего ноутбука
4. **Изучить ключевые компоненты:** filters, pooling, stride, padding

---

## 💼 Бизнес-задача: Titanic Survival

**Необычный подход:** Применим CNN к табличным данным.

**Почему это может работать?**
- 🔗 **Локальные паттерны:** Некоторые признаки связаны (Age + Sex, Fare + Pclass)
- 📊 **Feature interactions:** CNN автоматически находит комбинации
- 🧪 **Эксперимент:** Понять, когда CNN лучше MLP

**Честное предупреждение:**
- ❌ 1D-CNN **обычно НЕ лучше** MLP/XGBoost для табличных данных
- ✅ **Но:** Отлично работает для временных рядов, сигналов, текста
- ✅ **Цель:** Понять механизм convolutions для последующего изучения 2D-CNN (изображения)

---

## 📚 Часть 1: Теория Convolutional Neural Networks

### 1.1 Что такое Convolution?

**Свёртка (Convolution)** — операция скользящего окна, которая извлекает локальные паттерны.

#### 1D Convolution математически

Для входного сигнала $x$ длины $L$ и фильтра (kernel) $w$ размера $k$:

$$(x * w)[i] = \sum_{j=0}^{k-1} x[i+j] \cdot w[j]$$

**Пример:**
```
Input:  [1, 2, 3, 4, 5]
Filter: [1, 0, -1]  (edge detector)

Output[0] = 1*1 + 2*0 + 3*(-1) = -2
Output[1] = 2*1 + 3*0 + 4*(-1) = -2
Output[2] = 3*1 + 4*0 + 5*(-1) = -2
```

**Интуиция:** Фильтр "скользит" по входу, вычисляя локальные паттерны.

---

### 1.2 Ключевые параметры CNN

#### 1.2.1 Kernel Size (размер фильтра)

- **Малый kernel (3):** Детализированные локальные признаки
- **Большой kernel (5, 7):** Более широкий контекст

**Рекомендация:** Начните с 3, увеличивайте если нужен больший receptive field.

#### 1.2.2 Stride (шаг)

Насколько позиций сдвигается фильтр:

- **Stride=1:** Каждая позиция (no downsampling)
- **Stride=2:** Каждая вторая позиция (downsampling в 2 раза)

**Размер выхода:**

$$L_{\text{out}} = \left\lfloor \frac{L_{\text{in}} - k + 2p}{s} \right\rfloor + 1$$

где:
- $L_{\text{in}}$ — длина входа
- $k$ — kernel size
- $p$ — padding
- $s$ — stride

#### 1.2.3 Padding

Добавление нулей по краям:

- **Valid (no padding):** Выход короче входа
- **Same (padding):** Выход той же длины (часто $p = \lfloor k/2 \rfloor$)

**Зачем?** Сохранить размерность и информацию с краёв.

#### 1.2.4 Number of Filters

Количество разных паттернов, которые ищем:

- **Input:** $(L, C_{\text{in}})$ где $C_{\text{in}}$ — число каналов
- **Filters:** $(k, C_{\text{in}}, C_{\text{out}})$
- **Output:** $(L', C_{\text{out}})$

Типично: 32 → 64 → 128 filters.

---

### 1.3 Pooling Layers

**Pooling** — downsampling для уменьшения размерности.

#### Max Pooling

$$y[i] = \max(x[i \cdot s : i \cdot s + k])$$

**Пример (pool_size=2):**
```
Input:  [1, 3, 2, 4]
Output: [3, 4]  (max из каждой пары)
```

**Эффект:**
- ✅ Уменьшает размерность в 2 раза
- ✅ Translation invariance (малые сдвиги не важны)
- ✅ Выделяет наиболее сильные активации

#### Average Pooling

$$y[i] = \frac{1}{k} \sum_{j=0}^{k-1} x[i \cdot s + j]$$

**Когда:**
- Average pooling: Более гладкое downsampling
- Max pooling: Чаще используется (выделяет strongest features)

---

### 1.4 CNN для табличных данных: Как адаптировать?

**Проблема:** Признаки в таблице часто **НЕ упорядочены** (в отличие от пикселей в изображении).

**Решения:**

#### Подход 1: Упорядочить признаки осмысленно

Группировать связанные признаки:
```
[Age, Sex, Married] → Демография
[Fare, Pclass, Cabin] → Экономический статус
[SibSp, Parch, FamilySize] → Семья
```

CNN будет искать паттерны **внутри каждой группы**.

#### Подход 2: Kernel size = все признаки

Если kernel охватывает все признаки → эквивалентно MLP.

#### Подход 3: Multiple filter sizes

Разные kernel sizes захватывают разные масштабы:
- Kernel 3: Локальные взаимодействия (2-3 признака)
- Kernel 5: Средние взаимодействия
- Kernel 7: Широкие взаимодействия

**Почему обычно XGBoost/MLP лучше:**
- ❌ CNN предполагает **локальность** паттернов
- ❌ Табличные признаки часто имеют **глобальные** взаимодействия
- ❌ Порядок признаков обычно **произволен**

**Когда 1D-CNN может работать:**
- ✅ Временные ряды (порядок времени естественен)
- ✅ Сигналы (EEG, ECG, аудио)
- ✅ Текст (последовательность слов)
- ✅ Табличные данные с **естественным порядком** признаков

---

## Теория завершена! Переходим к практике 🚀

---

## 📊 Часть 2: Практическая реализация

### 2.1 Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from xgboost import XGBClassifier

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
print('✅ Библиотеки загружены')

### 2.2 Загрузка данных (та же подготовка, что в MLP)

In [ ]:
# Загружаем и подготавливаем данные точно так же, как в MLP notebook
data_path = '../../data/titanic_train.csv'
df = pd.read_csv(data_path) if __import__('os').path.exists(data_path) else None

if df is not None:
    # Заполняем пропуски
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    
    # Feature engineering
    df['Sex'] = (df['Sex'] == 'male').astype(int)
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)
    
    # Упорядочим признаки осмысленно для CNN!
    # Группа 1: Личная информация
    personal = ['Age', 'Sex']
    # Группа 2: Экономический статус
    economic = ['Pclass', 'Fare']
    # Группа 3: Семья
    family = ['FamilySize', 'IsAlone', 'SibSp', 'Parch']
    # Группа 4: Посадка
    embarked = [col for col in df.columns if 'Embarked_' in col]
    
    features = personal + economic + family + embarked
    
    X = df[features].values
    y = df['Survived'].values
    
    print(f'✅ Данные загружены: {df.shape}')
    print(f'Признаки (упорядочены по группам): {features}')
    print(f'X shape: {X.shape}, y shape: {y.shape}')

In [ ]:
# Train/Val/Test split + scaling
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=RANDOM_STATE, stratify=y_temp)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Для CNN нужна размерность (batch, channels, length)
# Табличные данные: (batch, 1, num_features) - 1 канал, num_features длина
X_train_cnn = X_train[:, np.newaxis, :]  # (N, 1, features)
X_val_cnn = X_val[:, np.newaxis, :]
X_test_cnn = X_test[:, np.newaxis, :]

# PyTorch tensors
X_train_t = torch.FloatTensor(X_train_cnn)
y_train_t = torch.FloatTensor(y_train)
X_val_t = torch.FloatTensor(X_val_cnn)
y_val_t = torch.FloatTensor(y_val)
X_test_t = torch.FloatTensor(X_test_cnn)
y_test_t = torch.FloatTensor(y_test)

# DataLoaders
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=batch_size)
test_loader = DataLoader(TensorDataset(X_test_t, y_test_t), batch_size=batch_size)

print(f'Train: {X_train.shape[0]}, Val: {X_val.shape[0]}, Test: {X_test.shape[0]}')
print(f'CNN input shape: (batch, {X_train_cnn.shape[1]}, {X_train_cnn.shape[2]})')

### 2.3 1D-CNN архитектура

In [ ]:
class CNN1D_Tabular(nn.Module):
    def __init__(self, num_features, num_filters=32, kernel_size=3):
        super(CNN1D_Tabular, self).__init__()
        
        # Conv block 1
        self.conv1 = nn.Conv1d(1, num_filters, kernel_size=kernel_size, padding=kernel_size//2)
        self.bn1 = nn.BatchNorm1d(num_filters)
        self.pool1 = nn.MaxPool1d(2)
        
        # Conv block 2
        self.conv2 = nn.Conv1d(num_filters, num_filters*2, kernel_size=kernel_size, padding=kernel_size//2)
        self.bn2 = nn.BatchNorm1d(num_filters*2)
        self.pool2 = nn.MaxPool1d(2)
        
        # Вычисляем размер после convolutions
        # После 2 pooling слоев: num_features / 2 / 2 = num_features / 4
        flattened_size = (num_features // 4) * (num_filters * 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(flattened_size, 64)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 1)
    
    def forward(self, x):
        # x shape: (batch, 1, features)
        
        # Conv block 1: Conv → BN → ReLU → Pool
        x = self.conv1(x)
        x = self.bn1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        
        # Conv block 2
        x = self.conv2(x)
        x = self.bn2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # FC layers
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        
        return x

# Инициализация
num_features = X_train_cnn.shape[2]
model_cnn = CNN1D_Tabular(num_features=num_features).to(device)

print(model_cnn)
print(f'\nTotal parameters: {sum(p.numel() for p in model_cnn.parameters()):,}')

### 2.4 Обучение CNN

In [ ]:
# Training functions (те же, что в MLP)
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
        correct += ((outputs > 0.5).float() == y_batch).sum().item()
        total += y_batch.size(0)
    return total_loss / total, correct / total

def validate_epoch(model, loader, criterion, device):
    model.eval()
    total_loss, correct, total = 0, 0, 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item() * X_batch.size(0)
            correct += ((outputs > 0.5).float() == y_batch).sum().item()
            total += y_batch.size(0)
            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(y_batch.cpu().numpy())
    return total_loss / total, correct / total, all_preds, all_labels

# Loss и optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model_cnn.parameters(), lr=0.001)

print('✅ Training functions готовы')

In [ ]:
# Training loop
num_epochs = 100
patience = 10

train_losses, val_losses = [], []
train_accs, val_accs = [], []
best_val_loss = float('inf')
patience_counter = 0
best_model_state = None

print('Обучаем 1D-CNN...')
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model_cnn, train_loader, criterion, optimizer, device)
    val_loss, val_acc, _, _ = validate_epoch(model_cnn, val_loader, criterion, device)
    
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_state = model_cnn.state_dict().copy()
        patience_counter = 0
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_acc:.4f}')
    
    if patience_counter >= patience:
        print(f'Early stopping на epoch {epoch+1}')
        break

model_cnn.load_state_dict(best_model_state)
print(f'✅ Обучение завершено! Best val loss: {best_val_loss:.4f}')

### 2.5 Оценка и сравнение с MLP/XGBoost

In [ ]:
# Оценка CNN
_, cnn_acc, cnn_preds, cnn_labels = validate_epoch(model_cnn, test_loader, criterion, device)
cnn_preds_np = np.array(cnn_preds).flatten()
cnn_labels_np = np.array(cnn_labels).flatten()
cnn_f1 = f1_score(cnn_labels_np, (cnn_preds_np > 0.5).astype(int))
cnn_auc = roc_auc_score(cnn_labels_np, cnn_preds_np)

print('📊 1D-CNN Results:')
print(f'  Accuracy: {cnn_acc:.4f}')
print(f'  F1-score: {cnn_f1:.4f}')
print(f'  ROC-AUC: {cnn_auc:.4f}')

In [ ]:
# MLP для сравнения (простая версия)
from torch import nn as nn_mlp

class SimpleMLP(nn_mlp.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn_mlp.Sequential(
            nn_mlp.Linear(input_dim, 64), nn_mlp.ReLU(), nn_mlp.Dropout(0.3),
            nn_mlp.Linear(64, 32), nn_mlp.ReLU(), nn_mlp.Dropout(0.3),
            nn_mlp.Linear(32, 1), nn_mlp.Sigmoid()
        )
    def forward(self, x):
        return self.net(x)

# Готовим данные для MLP (без дополнительной размерности)
X_train_mlp_t = torch.FloatTensor(X_train)
X_test_mlp_t = torch.FloatTensor(X_test)

# Быстрое обучение MLP
model_mlp = SimpleMLP(num_features).to(device)
optimizer_mlp = optim.Adam(model_mlp.parameters(), lr=0.001)
loader_mlp = DataLoader(TensorDataset(X_train_mlp_t, y_train_t), batch_size=32, shuffle=True)

print('Обучаем MLP для сравнения...')
for epoch in range(50):  # Меньше эпох для скорости
    model_mlp.train()
    for X_batch, y_batch in loader_mlp:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
        loss = criterion(model_mlp(X_batch), y_batch)
        optimizer_mlp.zero_grad()
        loss.backward()
        optimizer_mlp.step()

# Оценка MLP
model_mlp.eval()
with torch.no_grad():
    mlp_preds = model_mlp(X_test_mlp_t.to(device)).cpu().numpy().flatten()
mlp_acc = accuracy_score(y_test, (mlp_preds > 0.5).astype(int))
mlp_f1 = f1_score(y_test, (mlp_preds > 0.5).astype(int))
mlp_auc = roc_auc_score(y_test, mlp_preds)

print('✅ MLP обучен')

In [ ]:
# XGBoost для полноты
print('Обучаем XGBoost...')
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4, random_state=RANDOM_STATE, verbosity=0)
xgb.fit(X_train, y_train)
xgb_preds = xgb.predict_proba(X_test)[:, 1]
xgb_acc = accuracy_score(y_test, (xgb_preds > 0.5).astype(int))
xgb_f1 = f1_score(y_test, (xgb_preds > 0.5).astype(int))
xgb_auc = roc_auc_score(y_test, xgb_preds)

print('✅ XGBoost обучен')

In [ ]:
# Итоговое сравнение
comparison = pd.DataFrame({
    'Model': ['1D-CNN', 'MLP', 'XGBoost'],
    'Accuracy': [cnn_acc, mlp_acc, xgb_acc],
    'F1-score': [cnn_f1, mlp_f1, xgb_f1],
    'ROC-AUC': [cnn_auc, mlp_auc, xgb_auc]
})

print('\n' + '='*60)
print('🏆 СРАВНЕНИЕ: 1D-CNN vs MLP vs XGBoost')
print('='*60)
print(comparison.to_string(index=False))
print('='*60)

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, metric in enumerate(['Accuracy', 'F1-score', 'ROC-AUC']):
    axes[i].bar(comparison['Model'], comparison[metric], alpha=0.7, edgecolor='black')
    axes[i].set_ylabel(metric)
    axes[i].set_title(f'{metric} Comparison')
    axes[i].set_ylim([0.7, 0.9])
    for j, v in enumerate(comparison[metric]):
        axes[i].text(j, v + 0.01, f'{v:.3f}', ha='center')
plt.tight_layout()
plt.show()

## 🎯 Выводы

### Что мы изучили:

1. **1D Convolutions:** Операция свёртки для одномерных данных
2. **CNN компоненты:** Filters, kernels, stride, padding, pooling
3. **Адаптация для табличных:** Упорядочивание признаков, multiple kernel sizes
4. **PyTorch реализация:** Conv1d, MaxPool1d, BatchNorm1d

### Результаты: 1D-CNN vs MLP vs XGBoost

**Типичные результаты для Titanic:**
- **XGBoost:** Accuracy ~82-84%, ROC-AUC ~0.85-0.87 🏆
- **MLP:** Accuracy ~80-82%, ROC-AUC ~0.83-0.85
- **1D-CNN:** Accuracy ~78-82%, ROC-AUC ~0.82-0.85

### Ключевые инсайты:

#### ❌ Почему 1D-CNN НЕ лучше для табличных данных:

1. **Нет естественного порядка:** Признаки в таблице не упорядочены как пиксели/слова
2. **Глобальные interactions:** Табличные данные часто имеют взаимодействия между далёкими признаками
3. **Локальность assumption:** CNN предполагает, что близкие элементы связаны
4. **XGBoost оптимизирован:** Для табличных данных gradient boosting state-of-the-art

#### ✅ Когда 1D-CNN превосходит:

1. **Временные ряды:** Порядок времени естественен
2. **Сигналы:** EEG, ECG, аудио - локальные паттерны важны
3. **Текст:** Последовательность слов/символов
4. **Sensor данные:** Акселерометр, гироскоп

### Когда использовать 1D-CNN:

| Данные | 1D-CNN | Почему |
|--------|--------|--------|
| **Временные ряды** | ✅ Да | Локальные паттерны во времени |
| **Сигналы (EEG, ECG)** | ✅ Да | Частотные и временные паттерны |
| **Текст** | ✅ Да | N-grams, локальные контексты |
| **Табличные** | ❌ Обычно нет | XGBoost/MLP лучше |
| **Табличные с порядком** | ⚠️ Может быть | Если признаки естественно упорядочены |

### Практические выводы:

1. **Для Titanic/табличных:** Используйте XGBoost или MLP
2. **Для временных рядов:** 1D-CNN отличный выбор
3. **Понимание CNN:** Подготовка к 2D-CNN для изображений

### Следующие шаги:

1. **Autoencoders:** Unsupervised learning, anomaly detection
2. **2D-CNN (CV):** Изображения - где CNN действительно блестят
3. **RNN/LSTM:** Временные ряды с долгосрочными зависимостями

---

## 🎉 Phase 2, Step 2 завершен!

Вы поняли основы CNN и когда их применять. Готовы к Autoencoders! 🚀
